In [1]:
from pathlib import Path
from multiprocessing import Pool
import geopandas as gpd
from sentinel_helpers import scihub_band_paths

In [2]:
base_path = Path('input/tempelhofer_feld/')
input_files = list(base_path.glob('*.zip'))
area_of_interest = gpd.read_file(base_path / 'tempelhofer_feld.geojson')
ndvi_path = base_path / 'ndvi'

In [3]:
import rasterio as r
import rasterio.mask
import rasterio.plot as rplt
import numpy as np
from sentinel_helpers import scihub_normalize_range
from zipfile import BadZipFile

In [4]:
def calculate_ndvi(raster_file):
    global area_of_interest
    
    try:
        b04_path, b08_path = scihub_band_paths(raster_file, ['B04', 'B08'], '10m')
    except BadZipFile:
        print(f'Problem reading {raster_file}, skipping it')
        return

    with r.open(b04_path, 'r') as b04, r.open(b08_path, 'r') as b08:
        # we want to only write the bare minimum data necessary to disk
        out_meta = b04.meta.copy()

        # we reproject the geojson file we fetched above and convert it so that rasterio
        # can use it as a mask
        mask = area_of_interest.to_crs(out_meta['crs']).iloc[0].geometry
        miny, minx, maxy, maxx = mask.bounds

        # update the dimensions and save as geotiff, not jp2
        out_meta.update({
            'width': maxx - minx,
            'height': maxy - miny,
            'driver': 'GTiff',
            'dtype': 'float32'
        })    
        out_name = Path(b04_path).name.replace('B04', 'NDVI').replace('.jp2', '.tif')

        ! mkdir -p {ndvi_path}
        with r.open(ndvi_path / out_name, 'w+', **out_meta) as dst:
            # we take only the part out of our source raster that we actually need
            # crop=True clips off the borders
            b04, transform_b04 = rasterio.mask.mask(b04, shapes=[mask], crop=True)
            b08, _ = rasterio.mask.mask(b08, shapes=[mask], crop=True) # we ignore the returned transform because it's identical to the previous one

            b04 = scihub_normalize_range(b04).astype('f4') # <- f4 = float32
            b08 = scihub_normalize_range(b08).astype('f4')

            # uncomment the following line to see if your masked shape looks correct
            #rplt.show(b04, transform=transform_b04)
            #rplt.show(b08, transform=transform_b04)

            # we want to be able to ignore divide by zero errors so the formula is nicer to write
            np.seterr(divide='ignore', invalid='ignore')
            ndvi = (b08 - b04) / (b08 + b04)

            # uncomment the following line to see if we calculated the index correctly
            # rplt.show(ndvi, transform=transform_b04)

            dst.write(ndvi)

In [5]:
from tqdm.notebook import tqdm

In [6]:
%%time

pool = Pool()

for _ in tqdm(pool.imap_unordered(calculate_ndvi, input_files), total=len(input_files)):
    # this loop is only here so we can get the progress bar
    pass

Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190603T101031_N0212_R022_T33UUU_20190603T114652.zip, skipping it
Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190404T101031_N0211_R022_T32UQD_20190404T174806.zip, skipping it
Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190216T102111_N0211_R065_T33UUU_20190216T130428.zip, skipping it
Problem reading input/tempelhofer_feld/S2B_MSIL2A_20190419T101029_N0211_R022_T33UUU_20190419T132322.zip, skipping it
Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190407T102021_N0211_R065_T33UUU_20190407T134109.zip, skipping it
Problem reading input/tempelhofer_feld/S2B_MSIL2A_20190512T102029_N0212_R065_T33UUU_20190512T134103.zip, skipping it
Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190613T101031_N0212_R022_T33UUU_20190614T125329.zip, skipping itProblem reading input/tempelhofer_feld/S2A_MSIL2A_20190424T101031_N0211_R022_T32UQD_20190424T162325.zip, skipping it

Problem reading input/tempelhofer_feld/S2A_MSIL2A_20190822T10103

How many files could we process?

In [16]:
! ls -l {ndvi_path} | wc -l

31
